In [1]:
# Import relevant stuff
import pandas as pd
import numpy as np
from ast import literal_eval

In [2]:
# Goes from a list stored in a df to just the value inside.
def fix_list_format(df, col_title):
    fixed_row = []
    for index, row in df.iterrows():
        str1 = row[col_title]
        if len(str1) > 2:
            fixed_row.append(row[col_title][1:-1])
        else:
            fixed_row.append(np.nan)
    df[col_title] = fixed_row

In [3]:
# Load
df = pd.read_excel("C:/Users/Droor/OneDrive - Harvard University/Desktop/Master Folder/College (2022-2026)/Hoffman summer/main_notebook/only_stable_mp_matches_jenny.xlsx")
df = df.drop('material_dict_queried', axis=1)
df['material_dict'] = df['material_dict'].apply(literal_eval) #convert to list type
fix_list_format(df, 'unit_cell_volumes')
df.dtypes

sc_id                  object
supercon_formula       object
cleaned_formula        object
material_dict          object
highest_Tc             object
average_Tc            float64
std_Tc                float64
entry_count             int64
matches_by_element     object
matches_by_dict        object
mp_ids                 object
unit_cell_volumes      object
dtype: object

In [4]:
# Load element-data
element_data = pd.read_csv("C:/Users/Droor/OneDrive - Harvard University/Desktop/Master Folder/College (2022-2026)/Hoffman summer/main_notebook/element_data.csv")
element_data = element_data.drop("Unnamed: 0", axis=1)
numeric_columns = element_data.select_dtypes(include=np.number).columns
element_data = element_data[[element_data.columns[0]] + list(numeric_columns)]
element_data.head()

Element  AtomicMass  AtomicNumber  FirstIonizationEnergy  AtomicRadius  \
0       H    1.007940             1                 1311.3          53.0   
1      He    4.002602             2                 2361.3          31.0   
2      Li    6.941000             3                  519.9         167.0   
3      Be    9.012182             4                  898.8         112.0   
4       B   10.811000             5                  800.2          87.0   

   BoilingPoint  BrinellHardness  CovalentRadius    Density  \
0       -252.87              NaN              31     0.0899   
1       -268.93              NaN              28     0.1785   
2       1342.00              NaN             128   535.0000   
3       2470.00            600.0              96  1848.0000   
4       4000.00              NaN              85  2460.0000   

   ElectricalConductivity  ...   Resistivity  SoundSpeed  SpaceGroupNumber  \
0                     NaN  ...           NaN      1270.0             194.0   
1                     NaN  ...           NaN       970.0             225.0   
2            1.100000e+07  ...  9.400000e-08      6000.0             229.0   
3            2.500000e+07  ...  4.000000e-08     13000.0             194.0   
4            1.000000e-04  ...  1.000000e+04     16200.0             166.0   

   SpecificHeat  ThermalConductivity  ThermalExpansion  Valence  \
0       14300.0               0.1805               NaN        1   
1        5193.1               0.1513               NaN        0   
2        3570.0              85.0000          0.000046        1   
3        1820.0             190.0000          0.000011        2   
4        1030.0              27.0000          0.000006        3   

   VaporizationHeat  VolumeMagneticSusceptibility  SuperconductingPoint  
0             0.452                 -2.230000e-09                   NaN  
1             0.083                 -1.050000e-09                   NaN  
2           147.000                  1.370000e-05                   NaN  
3           297.000                 -2.328000e-05                 0.026  
4           507.000                 -2.140000e-05                   NaN  

[5 rows x 31 columns]

In [5]:
# Needs fixing: exploding and running the element-data-model.

import pandas as pd

# Explode the column with the list of dictionaries
df_expanded = df.explode('material_dict')
df_normalized = pd.json_normalize(df_expanded['material_dict'])
df_combined = pd.concat([df_expanded, df_normalized], axis=1)
df_combined.head()

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

#### **Hamidieh-style model using element data**

In [79]:
element_data = pd.read_csv("/content/gdrive/Shareddrives/Hoffman Lab SD/Research Projects/Machine Learning - Superconductors/11 Code/element_data.csv")
element_data = element_data.drop("Unnamed: 0", axis=1)
numeric_columns = element_data.select_dtypes(include=np.number).columns
element_data = element_data[[element_data.columns[0]] + list(numeric_columns)]
element_data.head()

---------------------------------------------------------------------------
FileNotFoundError                         Traceback (most recent call last)
Cell In[79], line 1
----> 1 element_data = pd.read_csv("/content/gdrive/Shareddrives/Hoffman Lab SD/Research Projects/Machine Learning - Superconductors/11 Code/element_data.csv")
      2 element_data = element_data.drop("Unnamed: 0", axis=1)
      3 numeric_columns = element_data.select_dtypes(include=np.number).columns

File C:\ProgramData\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311, in deprecate_nonkeyword_arguments.<locals>.decorate.<locals>.wrapper(*args, **kwargs)
    305 if len(args) > num_allow_args:
    306     warnings.warn(
    307         msg.format(arguments=arguments),
    308         FutureWarning,
    309         stacklevel=stacklevel,
    310     )
--> 311 return func(*args, **kwargs)

File C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\parsers\readers.py:680, in read_csv(filepath_or_buffer, sep, de

In [ ]:
df_exploded = pd.json_normalize(unique_mat_df['material_dict'])
unique_mat_df_exploded = pd.concat([unique_mat_df.drop('material_dict', axis=1), df_exploded], axis=1)
# unique_mat_df_exploded['n_atoms'] = unique_mat_df_exploded.iloc[:, 5:86].sum(axis=1)
unique_mat_df_exploded.head()
unique_mat_df_exploded.shape

In [ ]:
unique_mat_df_exploded.to_csv("exploded_element")

In [ ]:
max_keys = max(unique_mat_df['material_dict'], key=lambda x: len(x.keys()))

# Filter the DataFrame to include only the entries with the most keys
filtered_df = unique_mat_df[unique_mat_df['material_dict'].apply(lambda x: len(x.keys()) >= len(max_keys) - 2)]

# filtered_df.head(30)

In [ ]:
from numpy.ma.core import mean
from scipy.stats import gmean
from statsmodels.stats.weightstats import DescrStatsW

# We are going to create a 3D space with coordinates (material_name, property, feature)
data = np.empty((1, 30, 14070), dtype=object)
data.fill(None)

# Iterate over every *material*
for material_index, row in unique_mat_df_exploded.iterrows():

  # Get the formula/name of the material early
  material_formula = row[0]

  # Get the total number of atoms in a material.
  n_atoms = np.nansum(row[5:86])

  elem_row = row[5:86]

  # Create a Boolean mask for non-null entries
  mask = elem_row.notna()

  # Get the number of elements in a material:
  # n_elements = len(elem_row[mask])

  # Get intermediate variables to calculate features ---
  atom_name_list = []
  p_list = []

  for index1, value1 in elem_row[mask].iteritems():
    atom_name_list.append(index1)
    p_list.append((value1)/(n_atoms))

  # Iterate over every *property*
  for property_index, property_name in enumerate(element_data.columns[1:], start=1):

    t_list = []

    # Go through the atoms
    for index2, value2 in elem_row[mask].iteritems():

      # Find the row where 'Element' matches the value of index2
      element_row = element_data[element_data['Element'] == index2]

      # Extract the value from the specified column (property_name)
      property_value = element_row[property_name].values[0]

      # Append the property value to t_list
      t_list.append(property_value)

    p_list = np.array(p_list)
    t_list = np.array(t_list)

    # print(atom_name_list)

    # w_list = t_list / np.sum(t_list) # Could be wrong


    # list_features = ['mean', 'weighted_mean', 'geom_mean', 'weighted_geom_mean', 'entropy', 'range', 'weighted_range', 'std', 'weighted_std']

    # Add into our *3D-magic-thing* this coordinate. Problem may be that it expects indices, not names.
    data[0][property_index][material_index] = np.mean(t_list)

    # data[1][property_index][material_index] = sum(np.multiply(t_list, p_list))

    # data[2][property_index][material_index] = gmean(t_list)

    # data[3][property_index][material_index] = 1  # initialize it
    # for ti, pi in zip(t_list,p_list):
    #   data[3][property_index][material_index] *= ti ** pi

    # data[4][property_index][material_index] = 0  # initialize it
    # for wi in w_list:
    #   data[4][property_index][material_index] -=  wi*(np.log(wi))

    # # weighted_entropy

    # data[5][property_index][material_index] = np.ptp(t_list)

    # data[6][property_index][material_index] = np.ptp(np.multiply(t_list, p_list))

    # data[7][property_index][material_index] = np.std(t_list)

    # data[8][property_index][material_index] = DescrStatsW(t_list, weights=p_list, ddof=1).std


    # Can GPT to get better when done.


In [ ]:
from numpy import nanmean, nanstd, log, ptp
from scipy.stats import gmean

# We are going to create a 3D space with coordinates (material_name, property, feature)
data = np.empty((9, 36, 14070), dtype=object)
data.fill(None)

# Get intermediate variables to calculate features
elem_row = unique_mat_df_exploded.iloc[:, 5:86]
mask = elem_row.notna()
n_atoms = np.nansum(elem_row.values, axis=1)
n_elements = np.sum(mask, axis=1)
p_list = elem_row.values / n_atoms[:, np.newaxis]



# # Iterate over every material
# for material_index, row in unique_mat_df_exploded.iterrows():
#     material_formula = row[0]

#     t_list = element_data[element_data['Element'].isin(elem_row.loc[material_index][mask.loc[material_index]].index)][element_data.columns[1:]].values.flatten()
#     w_list = t_list / np.sum(t_list)

#     data[0, :, material_index] = nanmean(t_list, axis=0)
#     data[1, :, material_index] = np.dot(t_list, p_list[material_index])
#     data[2, :, material_index] = gmean(t_list)
#     data[3, :, material_index] = np.prod(np.power(t_list, p_list[material_index]))
#     data[4, :, material_index] = -np.sum(w_list * np.log(w_list))
#     data[5, :, material_index] = ptp(t_list)
#     data[6, :, material_index] = ptp(t_list * p_list[material_index])
#     data[7, :, material_index] = nanstd(t_list, ddof=1)
#     data[8, :, material_index] = DescrStatsW(t_list, weights=p_list[material_index], ddof=1).std


In [ ]:
# Example Series with NaN values
series = pd.Series([1, 2, np.nan, 4, 5, 6, 7, np.nan, 9, 10, np.nan, 12])

# Create a Boolean mask for non-null entries
mask = series.notna()

# Iterate over the non-null entries preserving the original index
for index, value in series[mask].iteritems():
    print(f"Index: {index}, Value: {value}")

In [ ]:
row = unique_mat_df_exploded.iloc[1]
row = row[5:86]
mask = row.notna()
print(len(row[mask]))
atom_name_list = []
p_list = []

for index1, value1 in elem_row[mask].iteritems():
  atom_name_list.append(index1)
  #p_list.append((value1)/(n_atoms))

  # Iterate over every *property*
  for property_index, property_name in enumerate(element_data.columns):
    print(property_name)
    print(property_index)
    t_list = []

    # Go through the atoms
    for index2, value2 in elem_row[mask].iteritems():

      # Find the row where 'Element' matches the value of index2
      element_row = element_data[element_data['Element'] == index2]

      # Extract the value from the specified column (property_name)
      property_value = element_row[property_name].values[0]

      # Append the property value to t_list
      t_list.append(property_value)

    p_list = np.array(p_list)
    t_list = np.array(t_list)

    print(t_list)
    w_list = t_list / np.sum(t_list) # Could be wrong

In [ ]:
t_list = np.array([1,2,3,3,4])
p_list = np.array([53,4,83,2,3,1])
print(t_list / np.sum(t_list))